## Fine tuning T5 for Seq2Seq classification using LoRA

In this Python notebook, we'll fine-tune a sequence-to-sequence (seq2seq) Large Language Model (LLM) for a text classification task. Our focus will be on the use of LoRA (Low-Rank Approximation), a method that allows us to adapt large models efficiently while preserving the original model's structure and knowledge.

The task at hand involves classifying text messages into spam or ham, a simple problem to demonstrate capabilities of LLMs. However, our main objective isn't to solve this classification problem but to explore and understand the process of fine-tuning a seq2seq LLM using LoRA. This involves adapting a pre-trained model to our specific task with minimal computational resources, while maintaining or even enhancing its performance.

We'll leverage several tools and technologies throughout this notebook:

- **Transformers Library**: Provides us with a vast collection of pre-trained models and utilities for working with them, including our model of choice, T5.
- **PyTorch**: A leading deep learning library that offers flexibility and a dynamic computation graph, making it ideal for our experimentation.
- **Pandas**: Essential for data manipulation and analysis, helping us to prepare our dataset for training.
- **LoRA** (Low-Rank Adaptation): The key to our fine-tuning process, allowing us to efficiently update a small subset of the model's parameters, specifically designed for adapting large language models to new tasks without extensive resources.

Let's dive deeper into each component.

Uncomment to install packages

In [ ]:
# !pip install -q transformers torch peft pandas accelerate

Import the necessary packages
**peft** (Parameter-efficient Fine-tuning): This library is used for getting LoRA config function as well as getting peft model.

Brief about PEFT - it involves adapting a pre-trained model to a new task by updating a small subset of its parameters. This method retains the original model's knowledge while efficiently achieving high performance on specific tasks, significantly reducing computational costs and time compared to full model fine-tuning.

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import torch
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("SMSSpamCollection.txt", sep = "\t", header=None)
df.columns = ["label", "sms"]
df

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


Selecting T5-small model for our task. You can select any other seq2seq model from huggingface.

In [3]:
model_name = "google-t5/t5-small"
d_map = "auto"

Loading tokenizer and model from huggingface

In [4]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, device_map = d_map)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
df["sms"][5567]

'This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.'

We have to pre-process our data in a format that is required in the downstream tasks. The seq2seq models like T5 expects these things to work, 
- input_ids, 
- labels 
- decoder_input_ids 

Here we will define our data in a list form (explained below), we have defined input_ids and label here, however, we have not defined the decoder_input_ids. That is because we will be using a Datacollator, which will take care of all the processing required for that. You can try to define it yourself as well, but I ran into some problems because of that. 

In [7]:
def process_data(df):
    data = []
    for i in range(len(df)):
        data.append({"input_ids": tokenizer.encode("classify: " + df["sms"][i], max_length=512, return_tensors="pt", truncation=True, padding="max_length").squeeze(),
                    #   "labels":tokenizer.encode(df["label"][i], max_length=512, return_tensors="pt", truncation=True, padding="max_length").squeeze(),
                      "labels":tokenizer.encode(df["label"][i], return_tensors="pt").squeeze(),
                    #   "decoder_input_ids":tokenizer.encode(df["label"][i], max_length=512, return_tensors="pt", truncation=True, padding="max_length").squeeze()
                      })
    return data

list_data = process_data(df)

The huggingface trainer model expects data to be in certain format. The data should be,
- Iterable 
- Should be able to slice batches of data

Due to this, we define our custom datasets aling with functions like __getitem__ and __len__. I ran into probelms while using Dataset library from huggingface, so I used torch's Dataset library.

In [8]:
class T5Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, key):
        return self.data[key]

    def __len__(self):
        return len(self.data)

dict_data = T5Dataset(list_data)


**DataCollatorForSeq2Seq** is a specialized data collator used with sequence-to-sequence (seq2seq) models for tasks like translation or summarization. 
- It dynamically pads your batch of examples to the length of the longest sequence,
- Prepares the decoder input IDs if necessary
- Handles other seq2seq specific preprocessing tasks

By using this collator, we ensure that our data is correctly formatted and optimized for training seq2seq models, leading to more efficient training sessions. It's particularly helpful because it abstracts away much of the manual preprocessing and ensures that the data fed into the model is uniform, allowing the model to train more effectively.

In [11]:
T5DataCollator  = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setting up and executing a training routine for a model using the PEFT (Parameter-Efficient Fine-Tuning) method. It splits the dataset, configures training arguments, applies LoRA settings, and then runs the training process.

Key parameters for Training Arguments:

- **output_dir**: Path to save model checkpoints.
- **auto_find_batch_size**: Enables automatic batch size determination.
- **learning_rate**: Adjusts the optimization step size for loss function minimization.
- **num_train_epochs**: Defines how many times the model will see the entire training set.
- **logging_steps & save_steps**: Determine logging and model saving frequency, respectively.

Key parameters for LoRA:

- **r & lora_alpha**: Define rank(approximation) and scale(can be understood how much influence we want for our current data, typical range (16-256)) for parameter updates, influencing training efficiency.
- **target_modules**: Identifies model parts like (query, key, value) for LoRA application, allowing focused updates.
- **lora_dropout**: Implements dropout for LoRA's parameters to aid in generalization.
- **task_type**: Specifies the model task, ensuring correct LoRA adaptation.

In [12]:
train_size = int(0.8 * len(dict_data))
val_size = len(dict_data) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dict_data, [train_size, val_size])


peft_training_args = TrainingArguments(
    output_dir="./fine-tuned-t6",
    auto_find_batch_size=True,
    learning_rate=1e-2, # Higher learning rate than full fine-tuning.
    num_train_epochs=3,
    logging_steps=100,
    # per_device_train_batch_size=4,
    save_steps=10,
    # max_steps=20_0
)

loraconfig = LoraConfig(
    r=64,
    lora_alpha = 64,
    target_modules=["q", "v"],
    lora_dropout=0.3,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

peft_model = get_peft_model(model,
                            loraconfig)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=train_dataset,
    eval_dataset =val_dataset,
    data_collator=T5DataCollator,
)

peft_trainer.train()

Step,Training Loss
100,0.663900
200,0.196900
300,0.177100
400,0.158300
500,0.190800
600,0.152700
700,0.127800
800,0.108600
900,0.110700
1000,0.088500


TrainOutput(global_step=1674, training_loss=0.14858525428863123, metrics={'train_runtime': 421.5954, 'train_samples_per_second': 31.715, 'train_steps_per_second': 3.971, 'total_flos': 1906564990500864.0, 'train_loss': 0.14858525428863123, 'epoch': 3.0})

Saving the finetuned model

In [15]:
peft_model_path="./fine-tuned-t7/model/"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./fine-tuned-t7/model/tokenizer_config.json',
 './fine-tuned-t7/model/special_tokens_map.json',
 './fine-tuned-t7/model/spiece.model',
 './fine-tuned-t7/model/added_tokens.json')

We then have to load the PEFT trained model using the peft library's PeftModel and use it to run inference.

In [16]:
peft_model = PeftModel.from_pretrained(model,
                                       './fine-tuned-t7/model/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False, device_map = "auto")

prom = "This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate."
input_ids = tokenizer.encode("classify: " +prom, return_tensors="pt", padding="max_length", max_length=512, truncation=True).to("cuda")

peft_model_outputs= peft_model.generate(input_ids = input_ids)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
peft_model_text_output

'spam'